In [13]:
import torch
import pyro
from pyro import poutine
from pyro import distributions as dist

In [ ]:
# Smoothed models

def smoothed_lda(data):
    alpha_prior = torch.ones(ntopics)
    nu_prior = torch.ones(vocab_size)
    with pyro.plate("topic_loop", ntopics):
        topics = pyro.sample("topics", dist.Dirichlet(nu_prior.repeat(ntopics, 1)))
    with pyro.plate("doc_loop", ndocs) as ind:
        data = data[:, ind]
        theta = pyro.sample("theta", dist.Dirichlet(alpha_prior))
        with pyro.plate("word_loop", nwords):
            pyro.sample("z", dist.Categorical(theta))
            pyro.sample("w", dist.Categorical(topics[z]), obs=data)


def smoothed_guide(data):
    lambda_q = pyro.param(
        "lambda_q", torch.ones(ntopics, vocab_size), constraint=positive
    )
    gamma_q = pyro.param(
        "gamma_q", torch.ones(ndocs, ntopics), constraint=positive
    )
    phi_q = pyro.param(
        "phi_q", torch.ones(nwords, ndocs, ntopics), constraint=positive
    )

    with pyro.plate("topic_loop", ntopics):
        topics = pyro.sample("topics", dist.Dirichlet(lambda_q))
    with pyro.plate("doc_loop", ndocs):
        theta = pyro.sample("theta", dist.Dirichlet(gamma_q))
        with pyro.plate("word_loop", nwords):
            z = pyro.sample("z", dist.Categorical(phi_q))
    return topics, theta, z

In [ ]:
from gensim.models.ldamodel import LdaModel
from gensim import corpora

id2word = dict(enumerate(set("|".join(data.apply("|".join)).split("|"))))
dictionary = corpora.Dictionary(data.tolist())
corpus = [dictionary.doc2bow(text) for text in data.tolist()]

NUM_TOPICS = 3
ldamodel = LdaModel(corpus, num_topics=NUM_TOPICS, id2word=id2word, passes=15)
ldamodel.save("model5.gensim")

topics = ldamodel.print_topics(num_words=5)
for topic in topics:
    print(topic)